In [1]:
import openeo
import pandas as pd

connection = openeo.connect(url="openeo.dataspace.copernicus.eu")
connection.authenticate_oidc()

Authenticated using refresh token.


<Connection to 'https://openeo.dataspace.copernicus.eu/openeo/1.1/' with OidcBearerAuth>

In [2]:
df = pd.read_csv('worldcities.csv')
e_df = df[df.country == 'Ethiopia'][4:8]

In [3]:
e_df

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
2186,Gonder,Gonder,12.6075,37.4592,Ethiopia,ET,ETH,Āmara,NaN,323900.0,1231234833
2188,Mekele,Mekele,13.4969,39.4769,Ethiopia,ET,ETH,Tigray,admin,323700.0,1231468531
2301,Āwasa,Awasa,7.0500,38.4667,Ethiopia,ET,ETH,YeDebub Bihēroch Bihēreseboch na Hizboch,admin,300100.0,1231663122
2490,Dire Dawa,Dire Dawa,9.6000,41.8667,Ethiopia,ET,ETH,Dirē Dawa,admin,277000.0,1231089714


In [4]:
from datetime import datetime

date_tuples = [
    (datetime(year, 5, 1).strftime('%Y-%m-%d'), datetime(year, 5, 31).strftime('%Y-%m-%d'))
    for year in range(2023, 2014, -1)
]

date_tuples[::-1]

[('2015-05-01', '2015-05-31'),
 ('2016-05-01', '2016-05-31'),
 ('2017-05-01', '2017-05-31'),
 ('2018-05-01', '2018-05-31'),
 ('2019-05-01', '2019-05-31'),
 ('2020-05-01', '2020-05-31'),
 ('2021-05-01', '2021-05-31'),
 ('2022-05-01', '2022-05-31'),
 ('2023-05-01', '2023-05-31')]

In [5]:
class City:
    def __init__(self, name, lat, lng):
        self.name = name
        self.lat = lat
        self.lng = lng

    def get_spatial_extent(self, radius=0.2):
        spatial_extent = {
            "west": self.lng-radius, 
            "south": self.lat-radius, 
            "east": self.lng+radius, 
            "north": self.lat+radius
        }
        return spatial_extent

    def get_features(self, radius=0.15):
        features = {
            "type": "FeatureCollection", 
            "features": [
                {
                    "type": "Feature", 
                    "properties": {},
                    "geometry": {
                        "type": "Polygon", 
                        "coordinates": [
                            [
                                [self.lat - radius, self.lng - radius], 
                                [self.lat + radius, self.lng - radius], 
                                [self.lat + radius, self.lng + radius], 
                                [self.lat - radius, self.lng + radius], 
                                [self.lat - radius, self.lng - radius]
                            ]
                        ]
                    }
                }
            ]
        }
        return features

In [6]:
cities_input = {'city': [], 'coordinates': [] }

for idx, row in e_df.iterrows():
    cities_input['city'].append(row['city'])
    cities_input['coordinates'].append(City(row['city'], row['lat'], row['lng']).get_spatial_extent(radius=0.08))

city_coordinates = pd.DataFrame(cities_input)

In [7]:
def send_job(cube, title):
    res = []
    job  = cube.send_job(title=title)
    job_id = job.job_id
    results = job.start_and_wait().get_results()
    assets = results.get_assets()
    for asset in assets:
        res.append(asset.metadata)
    return res

In [10]:
#jobs_results = {"city": [], "date_range": [], "job_res_ndvi": [],"job_res_ndwi": []}

In [11]:
counter = 0

for date_range in date_tuples:
    
    start = date_range[0]
    end = date_range[1]
    
    for idx, row in city_coordinates.iterrows():

        if len(jobs_results['job_res_ndwi']) > counter:
            counter += 1
            continue
        else:
            for key in jobs_results.keys():
                jobs_results[key] = jobs_results[key][:counter]
        
        cube = connection.load_collection(
            "SENTINEL2_L1C",
            bands=["B04", "B08", "B03", "B11"],
            temporal_extent=(start, end),
            spatial_extent=row['coordinates'],
            max_cloud_cover=30
        )
        
        ndvi_cube = (cube.band('B08') - cube.band('B04')) / (cube.band('B08') + cube.band('B04'))
        ndwi_cube = (cube.band('B03') - cube.band('B11')) / (cube.band('B03') + cube.band('B11'))

        jobs_results["city"].append(row['city'])
        jobs_results["date_range"].append(date_range)
        jobs_results["job_res_ndvi"].append(send_job(ndvi_cube, "ndvi"))
        jobs_results["job_res_ndwi"].append(send_job(ndwi_cube, "ndwi"))

        counter += 1

/var/folders/j9/62n40sf93wbf8lw3sm2rxj6w0000gq/T/ipykernel_9772/2867525309.py:3: UserDeprecationWarning: Call to deprecated method create_job. (Usage of this legacy method is deprecated. Use `.create_job` instead.) -- Deprecated since version 0.10.0.
  job  = cube.send_job(title=title)


0:00:00 Job 'j-231104b95ac44ea5ab3f54e620a2943a': send 'start'
0:00:29 Job 'j-231104b95ac44ea5ab3f54e620a2943a': created (progress N/A)
0:00:36 Job 'j-231104b95ac44ea5ab3f54e620a2943a': created (progress N/A)
0:00:44 Job 'j-231104b95ac44ea5ab3f54e620a2943a': created (progress N/A)
0:00:52 Job 'j-231104b95ac44ea5ab3f54e620a2943a': running (progress N/A)
0:01:02 Job 'j-231104b95ac44ea5ab3f54e620a2943a': running (progress N/A)
0:01:17 Job 'j-231104b95ac44ea5ab3f54e620a2943a': running (progress N/A)
0:01:32 Job 'j-231104b95ac44ea5ab3f54e620a2943a': running (progress N/A)
0:01:52 Job 'j-231104b95ac44ea5ab3f54e620a2943a': running (progress N/A)
0:02:16 Job 'j-231104b95ac44ea5ab3f54e620a2943a': running (progress N/A)
0:02:46 Job 'j-231104b95ac44ea5ab3f54e620a2943a': running (progress N/A)
0:03:23 Job 'j-231104b95ac44ea5ab3f54e620a2943a': finished (progress N/A)
0:00:00 Job 'j-23110454b85a4e6bb8a606cd51aca0e5': send 'start'
0:00:24 Job 'j-23110454b85a4e6bb8a606cd51aca0e5': created (progress N/

JobFailedException: Batch job 'j-231104219d36442884cd3fd62345dbfe' didn't finish successfully. Status: error (after 0:01:58).

In [23]:
connection.job('j-231104219d36442884cd3fd62345dbfe').logs()

[{'id': '[1699110930903, 1488387]',
  'time': '2023-11-04T15:15:30.903Z',
  'level': 'info',
  'message': "EJR creating job_id='j-231104219d36442884cd3fd62345dbfe' created='2023-11-04T15:15:30Z'"},
 {'id': '[1699110930903, 1488840]',
  'time': '2023-11-04T15:15:30.903Z',
  'level': 'debug',
  'message': 'EJR Request `POST /jobs`: start 2023-11-04 15:15:30.902943'},
 {'id': '[1699110930903, 1489256]',
  'time': '2023-11-04T15:15:30.903Z',
  'level': 'debug',
  'message': "Doing EJR request `POST https://jobregistry.openeo.vito.be/jobs` headers.keys()=dict_keys(['Authorization'])"},
 {'id': '[1699110931195, 1518648]',
  'time': '2023-11-04T15:15:31.195Z',
  'level': 'debug',
  'message': 'EJR response on `POST /jobs`: 201'},
 {'id': '[1699110931195, 1519048]',
  'time': '2023-11-04T15:15:31.195Z',
  'level': 'debug',
  'message': 'EJR Request `POST /jobs`: end 2023-11-04 15:15:31.195538, elapsed 0:00:00.292595'},
 {'id': '[1699110931503, 1553315]',
  'time': '2023-11-04T15:15:31.503Z',
  'level': 'info',
  'message': "EJR get job data job_id='j-231104219d36442884cd3fd62345dbfe'"},
 {'id': '[1699110931503, 1553740]',
  'time': '2023-11-04T15:15:31.503Z',
  'level': 'debug',
  'message': 'Doing search with query {"query": {"bool": {"filter": [{"term": {"backend_id": "cdse-prod"}}, {"term": {"job_id": "j-231104219d36442884cd3fd62345dbfe"}}]}}, "_source": ["status", "*", "created", "job_id", "updated", "user_id"]}'},
 {'id': '[1699110931504, 1554365]',
  'time': '2023-11-04T15:15:31.504Z',
  'level': 'debug',
  'message': 'EJR Request `POST /jobs/search`: start 2023-11-04 15:15:31.503923'},
 {'id': '[1699110931504, 1554789]',
  'time': '2023-11-04T15:15:31.504Z',
  'level': 'debug',
  'message': "Doing EJR request `POST https://jobregistry.openeo.vito.be/jobs/search` headers.keys()=dict_keys(['Authorization'])"},
 {'id': '[1699110931600, 1563532]',
  'time': '2023-11-04T15:15:31.600Z',
  'level': 'debug',
  'message': 'EJR response on `POST /jobs/search`: 200'},
 {'id': '[1699110931600, 1563939]',
  'time': '2023-11-04T15:15:31.600Z',
  'level': 'debug',
  'message': 'EJR Request `POST /jobs/search`: end 2023-11-04 15:15:31.600365, elapsed 0:00:00.096442'},
 {'id': '[1699110931623, 1566412]',
  'time': '2023-11-04T15:15:31.623Z',
  'level': 'info',
  'message': "Starting job 'j-231104219d36442884cd3fd62345dbfe' from user User('9f07fc24-5d71-4ea6-8333-935307c9610d', {'oidc_userinfo': {'sub': '9f07fc24-5d71-4ea6-8333-935307c9610d', 'email_verified': True, 'name': 'Krzysztof Spalinski', 'preferred_username': 'krzysztof@spalinski.eu', 'given_name': 'Krzysztof', 'family_name': 'Spalinski', 'email': 'krzysztof@spalinski.eu'}}) (proxy user None)"},
 {'id': '[1699110931635, 1571706]',
  'time': '2023-11-04T15:15:31.635Z',
  'level': 'info',
  'message': "EJR get job data job_id='j-231104219d36442884cd3fd62345dbfe'"},
 {'id': '[1699110931635, 1572132]',
  'time': '2023-11-04T15:15:31.635Z',
  'level': 'debug',
  'message': 'Doing search with query {"query": {"bool": {"filter": [{"term": {"backend_id": "cdse-prod"}}, {"term": {"job_id": "j-231104219d36442884cd3fd62345dbfe"}}]}}, "_source": ["status", "*", "created", "job_id", "updated", "user_id"]}'},
 {'id': '[1699110931635, 1572758]',
  'time': '2023-11-04T15:15:31.635Z',
  'level': 'debug',
  'message': 'EJR Request `POST /jobs/search`: start 2023-11-04 15:15:31.635824'},
 {'id': '[1699110931636, 1573183]',
  'time': '2023-11-04T15:15:31.636Z',
  'level': 'debug',
  'message': "Doing EJR request `POST https://jobregistry.openeo.vito.be/jobs/search` headers.keys()=dict_keys(['Authorization'])"},
 {'id': '[1699110931752, 1577795]',
  'time': '2023-11-04T15:15:31.752Z',
  'level': 'debug',
  'message': 'EJR response on `POST /jobs/search`: 200'},
 {'id': '[1699110931752, 1578201]',
  'time': '2023-11-04T15:15:31.752Z',
  'level': 'debug',
  'message': 'EJR Request `POST /jobs/search`: end 2023-11-04 15:15:31.752723, elapsed 0:00:00.116899'},
 {'id': '[1699110931758, 1579051]',
  'time': '2023

In [13]:
print(len(jobs_results['city']),
     len(jobs_results['date_range']),
    len(jobs_results['job_res_ndvi']))

27 27 26


In [20]:
jobs_results_parsed = {
    'city': jobs_results['city'][:26],
    'date_range': jobs_results['date_range'][:26],
    'job_res_ndvi': jobs_results['job_res_ndvi'][:26],
    'job_res_ndwi': jobs_results['job_res_ndwi'][:26]
}

In [21]:
pd.DataFrame(jobs_results_parsed).to_csv("results_ethiopia_4_8_02_2016_2023.csv", index=False)

In [22]:
pd.DataFrame(jobs_results_parsed)

,city,date_range,job_res_ndvi,job_res_ndwi
0,Gonder,"(2023-05-01, 2023-05-31)","[{'file:nodata': ['nan'], 'href': 'https://ope...","[{'file:nodata': ['nan'], 'href': 'https://ope..."
1,Mekele,"(2023-05-01, 2023-05-31)","[{'file:nodata': ['nan'], 'href': 'https://ope...","[{'file:nodata': ['nan'], 'href': 'https://ope..."
2,Āwasa,"(2023-05-01, 2023-05-31)","[{'file:nodata': ['nan'], 'href': 'https://ope...","[{'file:nodata': ['nan'], 'href': 'https://ope..."
3,Dire Dawa,"(2023-05-01, 2023-05-31)","[{'file:nodata': ['nan'], 'href': 'https://ope...","[{'file:nodata': ['nan'], 'href': 'https://ope..."
4,Gonder,"(2022-05-01, 2022-05-31)","[{'file:nodata': ['nan'], 'href': 'https://ope...","[{'file:nodata': ['nan'], 'href': 'https://ope..."
5,Mekele,"(2022-05-01, 2022-05-31)","[{'file:nodata': ['nan'], 'href': 'https://ope...","[{'file:nodata': ['nan'], 'href': 'https://ope..."
6,Āwasa,"(2022-05-01, 2022-05-31)","[{'file:nodata': ['nan'], 'href': 'https://ope...","[{'file:nodata': ['nan'], 'href': 'https://ope..."
7,Dire Dawa,"(2022-05-01, 2022-05-31)","[{'file:nodata': ['nan'], 'href': 'https://ope...","[{'file:nodata': ['nan'], 'href': 'https://ope..."
8,Gonder,"(2021-05-01, 2021-05-31)","[{'file:nodata': ['nan'], 'href': 'https://ope...","[{'file:nodata': ['nan'], 'href': 'https://ope..."
9,Mekele,"(2021-05-01, 2021-05-31)","[{'file:nodata': ['nan'], 'href': 'https://ope...","[{'file:nodata': ['nan'], 'href': 'https://ope..."


In [24]:
connection.load_collection?

Signature:
connection.load_collection(
    collection_id: 'Union[str, Parameter]',
    spatial_extent: 'Optional[Dict[str, float]]' = None,
    temporal_extent: 'Union[Sequence[InputDate], Parameter, str, None]' = None,
    bands: 'Union[None, List[str], Parameter]' = None,
    properties: 'Optional[Dict[str, Union[str, PGNode, Callable]]]' = None,
    max_cloud_cover: 'Optional[float]' = None,
    fetch_metadata=True,
) -> 'DataCube'
Docstring:
Load a DataCube by collection id.

:param collection_id: image collection identifier
:param spatial_extent: limit data to specified bounding box or polygons
:param temporal_extent: limit data to specified temporal interval.
    Typically, just a two-item list or tuple containing start and end date.
    See :ref:`filtering-on-temporal-extent-section` for more details on temporal extent handling and shorthand notation.
:param bands: only add the specified bands
:param properties: limit data by metadata property predicates
:param max_cloud_cover: 